In [1]:
#install.packages("dils", repos = "http://cran.itam.mx/")
#install.packages("igraph", repos = "http://cran.itam.mx/")

library( "dils" )
library( "igraph" )

Warning message:
: package 'dils' was built under R version 3.3.1Loading required package: igraph

Attaching package: 'igraph'

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union

Loading required package: Rcpp


In [16]:
#edgeList <- read.table("red.final.sanos.txt", header = TRUE, sep = "\t", stringsAsFactors = F)
edgeList <- read.table("red.final.enfermos.txt", header = TRUE, sep = "\t", stringsAsFactors = F)

#Calculates number of edges
nEdges <- length(edgeList[,1])
nEdges

[1] 37862

In [17]:
#Makes a list using all the mRNA and genes (nodes constr.)
x <- sort( unique( c(edgeList[,1], edgeList[,2])) )

#Separate all genes from mRNA
y <- x[grepl("^[^(hsa)].+$", x)]
z <- x[grepl("hsa.*", x)]

#Joins them to preserve order
x <- c(y,z)

#Determinates number of nodes
nNde <- length(x)
nGene <- length(y)
nmRNA <- length(z)

nNde; nGene; nmRNA

[1] 4544

[1] 4033

[1] 511

In [18]:
#Creates a matrix (future adjacency matrix)
mat  <- matrix(0, nrow = nNde, ncol = nNde)

#Each col and row correspond to a gene or mRNA (stored in x)
rownames( mat ) <- x
colnames( mat ) <- rownames( mat )

In [19]:
#Makes a dictionary to asign each element in our adjacency list to our matrix.
dict <- vector(mode = "list", length = nNde )
names(dict) <- x

for (i in 1:nNde){ dict[[i]] <- i }

for (i in 1:nEdges){
    mat[ dict[[ edgeList[i,1] ]], dict[[ edgeList[i,2] ]] ] <- edgeList[i,3]
    mat[ dict[[ edgeList[i,2] ]], dict[[ edgeList[i,1] ]] ] <- edgeList[i,3]
}

In [20]:
#Checks if we did everything right
nEdges == sum(mat)/2
all(mat == t(mat))
isSymmetric(mat)

[1] FALSE

[1] TRUE

[1] TRUE

In [21]:
G <- graph.adjacency( adjmatrix = mat, mode = "undirected", diag = F, weighted = TRUE )

In [22]:
centralityEgn <- eigen_centrality(graph = G)
write.table( centralityEgn$vector, "eigenCentrality.txt", sep = "\t", col.names = F, quote = F)

In [23]:
centralityBtw <- betweenness(graph = G, v = V(G))
write.table( centralityBtw, "betweenness.txt", sep = "\t", col.names = F, quote = F)

In [24]:
centralityCls <- closeness(graph = G, v = V(G))
write.table( centralityCls, "closeness.txt", sep = "\t", col.names = F, quote = F)

In [25]:
centralityDgr <- degree(graph = G, v = V(G))
write.table( centralityDgr, "degree.txt", sep = "\t", col.names = F, quote = F)

In [26]:
#Split data in 3 graphs
matG <- mat[ 1:nGene, 1:nGene ]
colnames(matG) = rownames(matG) = y

matR <- mat[ (nGene+1):(nGene+nmRNA) , (nGene+1):(nGene+nmRNA) ]
colnames(matR) = rownames(matR) = z

matB <- matrix(0, length(x), length(x))
rownames(matB) = colnames(matB) = x

for(i in 1:nGene){ 
    for(j in (nGene+1):(nGene+nmRNA)){
        if( mat[i,j] > 0 ){ 
            matB[i,j] = matB[j,i] = mat[i,j]
} } }

In [27]:
#Applies every measure restricted to genes

genesG <- graph.adjacency( adjmatrix = matG, mode = "undirected", diag = F, weighted = TRUE)

centralityEgn <- eigen_centrality(graph = genesG)
write.table( centralityEgn$vector, "eigenCentrality_genes.txt", sep = "\t", col.names = F, quote = F)

centralityBtw <- betweenness(graph = genesG, v = V(genesG))
write.table( centralityBtw, "betweenness_genes.txt", sep = "\t", col.names = F, quote = F)

centralityCls <- closeness(graph = genesG, v = V(genesG))
write.table( centralityCls, "closeness_genes.txt", sep = "\t", col.names = F, quote = F)

centralityDgr <- degree(graph = genesG, v = V(genesG))
write.table( centralityDgr, "degree_genes.txt", sep = "\t", col.names = F, quote = F)

In [28]:
#Applies every measure restricted to miRNA

mrnaG <- graph.adjacency( adjmatrix = matR, mode = "undirected", diag = F, weighted = TRUE )

centralityEgn <- eigen_centrality(graph = mrnaG)
write.table( centralityEgn$vector, "eigenCentrality_mRNA.txt", sep = "\t", col.names = F, quote = F)

centralityBtw <- betweenness(graph = mrnaG, v = V(mrnaG))
write.table( centralityBtw, "betweenness_mRNA.txt", sep = "\t", col.names = F, quote = F)

centralityCls <- closeness(graph = mrnaG, v = V(mrnaG))
write.table( centralityCls, "closeness_mRNA.txt", sep = "\t", col.names = F, quote = F)

centralityDgr <- degree(graph = mrnaG, v = V(mrnaG))
write.table( centralityDgr, "degree_mRNA.txt", sep = "\t", col.names = F, quote = F)

In [29]:
#Applies every measure restricted to the bipartite graph

bipG <- graph.adjacency( adjmatrix = matB, mode = "undirected", diag = F, weighted = TRUE )

centralityEgn <- eigen_centrality(graph = bipG)
write.table( centralityEgn$vector, "eigenCentrality_bipartite.txt", sep = "\t", col.names = F, quote = F)

centralityBtw <- betweenness(graph = bipG, v = V(bipG))
write.table( centralityBtw, "betweenness_bipartite.txt", sep = "\t", col.names = F, quote = F)

centralityCls <- closeness(graph = bipG, v = V(bipG))
write.table( centralityCls, "closeness_bipartite.txt", sep = "\t", col.names = F, quote = F)

centralityDgr <- degree(graph = bipG, v = V(bipG))
write.table( centralityDgr, "degree_bipartite.txt", sep = "\t", col.names = F, quote = F)